<a href="https://colab.research.google.com/github/mjooee/Lietector-/blob/models/word_embedding_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#패키지 연결 (학습때는 오류 생겨서 설치 새로 하기)
import os, sys
my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' ## 패키지가 저장될 경로

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

GloVe

In [ ]:
!pip install glove-python3


In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tqdm import tqdm

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
#불용어 설정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '께', '네', '제', '그리고', '씨', '라고', '요', '을', '에서', '및', '그래서', '그렇다']

#토크나이저
okt = Okt()

In [ ]:
#불용어 저장
file_path = "/content/drive/MyDrive/stopwords"
with open(file_path, "w") as file:
    for word in stopwords:
        file.write(word + "\n")

In [ ]:
def PreProcessing(docName):

  path = root_dir + '/' + docName

  targetTXT = open(path, 'r')
  lines = targetTXT.readlines()
  targetTXT.close()

  removed_data = []

  for line in lines:
    # 한글 아닌 부분 제거
    s = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", line)
    if s == '': continue
    removed_data.append(s)

  tokenized_data = []
  for sentence in tqdm(removed_data):
      tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
      stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
      tokenized_data.append(stopwords_removed_sentence)
  return tokenized_data

In [ ]:
import re
import os

root_dir = '/content/drive/MyDrive/금융감독원'

tokenized_data = []

total_data = os.listdir(root_dir)

for file in total_data:
  tokenized_data.append(PreProcessing(file))



100%|██████████| 91/91 [00:00<00:00, 262.46it/s]
0it [00:00, ?it/s]
100%|██████████| 6/6 [00:00<00:00, 259.51it/s]
0it [00:00, ?it/s]
100%|██████████| 647/647 [00:01<00:00, 446.05it/s]


In [ ]:
#데이터 학습용, 테스트용 분리하기
num_of_train = int(len(tokenized_data) * 0.9) #학습용 데이터 434
num_of_test = int(len(tokenized_data) - num_of_train) #테스트용 데이터 49
train_data = tokenized_data[:num_of_train]
test_data = tokenized_data[num_of_train:]

In [ ]:
from glove import Corpus, Glove
corpus = Corpus()

/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [ ]:
#train_data 하나로 합치기
result = []

for doc in train_data:
  for line in doc:
    result.append(line)

In [ ]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)


['민원', '신고']
['금융', '민원', '불법', '금융', '신고', '보이스피싱', '옴부', '즈', '만', '고충', '민원', '등', '민원', '신고', '서비스', '제공']
['참여', '소통']


In [ ]:
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [ ]:
print(glove.most_similar("거짓말",10))

[('전하', 0.8151714885516751), ('네버', 0.8060414907896879), ('오기', 0.8049016603296133), ('신용정보', 0.8047584446592201), ('깔다', 0.7964809186293293), ('따다', 0.7950284310019733), ('입구', 0.792943132485455), ('걸어가다', 0.7918871286636268), ('부시', 0.7911487758720012)]


In [ ]:
print(glove.most_similar("보험",10))

[('재산', 0.9053570499619481), ('신청서', 0.895611243423162), ('채권', 0.7496437101983199), ('가짜', 0.716508193566304), ('코로나', 0.708526259703547), ('홈페이지', 0.6961024431140683), ('키트', 0.6912844006813798), ('엄마', 0.6904487672421498), ('추진', 0.6892084879934549)]


In [ ]:
#모델 저장
model_dir = '/content/drive/MyDrive'
glove.save(model_dir + '/glove.model')

In [ ]:
#모델 불러오기
from glove import Corpus, Glove

model_dir = '/content/drive/MyDrive'
glove = Glove.load(model_dir + '/glove.model')

Word2Vec

In [ ]:
import os
import re
from gensim.models import Word2Vec

# Word2Vec 모델 생성
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=1, workers=4)

# 단어 집합 생성
model.build_vocab(result)

# Word2Vec 모델 훈련
model.train(result, total_examples=len(result), epochs=10)

(2630689, 3206360)

In [ ]:
model.wv.most_similar(positive=[""], topn=10)

In [ ]:
model.wv.most_similar(positive=["보험"], topn=10)

문서 유사도 측정을 위한 평균 벡터 구하기

In [ ]:
#벡터 확인
glove.word_vectors[glove.dictionary['문자']]

array([ 0.12849111,  0.03125983,  0.10654665,  0.10092125,  0.01368536,
        0.14953994,  0.21666274, -0.07949901, -0.03180513, -0.08375887,
       -0.02531525, -0.05311906, -0.04103949,  0.01223502,  0.04243079,
       -0.0132026 ,  0.03088036, -0.08016305, -0.08873329, -0.03376111,
        0.11148858,  0.1541337 , -0.08586841, -0.03133955,  0.15573494,
        0.33979651,  0.06476719,  0.07664148, -0.14170417,  0.18929707,
       -0.12139163,  0.17127726, -0.08202848, -0.16979579, -0.00570018,
        0.03248315,  0.03185817, -0.10591522,  0.14521766,  0.0943319 ,
       -0.00126285, -0.341471  , -0.04784588,  0.00515481,  0.09446817,
        0.25890176,  0.05765107, -0.09662938, -0.0912614 ,  0.04172399,
       -0.11820596,  0.02833324, -0.00822468,  0.05761417, -0.08907522,
        0.14627459,  0.25548094, -0.06929996, -0.16034196, -0.08700238,
        0.05417271, -0.12081968,  0.04322855, -0.15139095, -0.13337305,
        0.12039159, -0.03998714,  0.08474986,  0.03721074, -0.11

In [ ]:
#벡터 평균으로 문서 벡터 구하기
def get_document_vectors_g(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for lines in document_list:
        doc2vec = None
        count = 0
        for line in lines:
          for word in line:
            if word in glove.dictionary.keys():
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = glove.word_vectors[glove.dictionary[word]]
                else:
                    doc2vec = doc2vec + glove.word_vectors[glove.dictionary[word]]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list


In [ ]:
word_vocab= list(model.wv.key_to_index.keys())
def get_document_vectors_w(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for lines in document_list:
      doc2vec = None
      count = 0
      for line in lines:
         for word in line:
            if word in word_vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model.wv.get_vector(word, norm=True)
                else:
                    doc2vec = doc2vec + model.wv.get_vector(word, norm=True)

      if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list



In [ ]:
#일반 대화 텍스트 데이터 전처리
root_dir = '/content/drive/MyDrive/일반대화'

normal_data = os.listdir(root_dir)

for file in normal_data:
  test_data.append(PreProcessing(file))

100%|██████████| 16/16 [00:00<00:00, 1165.51it/s]


In [ ]:
glove_embedding_list = get_document_vectors_g(test_data)
#w2v_embedding_list = get_document_vectors_w(test_data)
print('glove 문서 벡터의 수 :',len(glove_embedding_list)) #0~48: 보이스피싱 49~97: 일반 대화
#print('word2vec 문서 벡터의 수 :',len(w2v_embedding_list)) #0~48: 보이스피싱 49~97: 일반 대화

glove 문서 벡터의 수 : 49


In [ ]:
#문서 간의 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

glove_cosine_similarities = cosine_similarity(glove_embedding_list, glove_embedding_list)
w2v_cosine_similarities = cosine_similarity(w2v_embedding_list, w2v_embedding_list)

In [ ]:
glove_embedding_list[48]

array([-0.06096694,  0.05364959,  0.07719169, -0.00133167,  0.03457507,
       -0.02224125,  0.14256764,  0.00171454, -0.09389049,  0.03119496,
        0.04358822, -0.03601168, -0.0741869 , -0.03409515,  0.04719773,
        0.06983666,  0.06041908,  0.04222832,  0.00502249,  0.11280785,
        0.09987138,  0.06009712,  0.00628384,  0.12884026, -0.08806709,
        0.08021721,  0.10476244, -0.08869657, -0.01926132,  0.07485976,
        0.02593293, -0.01860019, -0.03775865, -0.05722962, -0.0731625 ,
       -0.06266245, -0.03760076, -0.02430847, -0.0638581 , -0.11080498,
        0.01099018,  0.01627234,  0.00834929,  0.03394186,  0.05533448,
        0.05633743,  0.02380877, -0.01659452, -0.01486328, -0.05116294,
       -0.0011934 , -0.06025851,  0.00636914, -0.02227506, -0.01055644,
       -0.01955189,  0.0509993 , -0.0107252 , -0.05301579, -0.11321575,
       -0.00948071, -0.02881999,  0.04531395, -0.02095783, -0.0227761 ,
        0.03534603, -0.0804168 ,  0.08685874,  0.06117865, -0.04

In [ ]:
#문서 간의 유사도 샘플
num = 49

glove_scores = list(enumerate(glove_cosine_similarities[num]))
w2v_scores = list(enumerate(w2v_cosine_similarities[num]))

glove_scores = sorted(glove_scores, key = lambda x: x[1], reverse = True)
glove_scores = glove_scores[1:6]

w2v_scores = sorted(w2v_scores, key = lambda x: x[1], reverse = True)
w2v_scores = w2v_scores[1:6]


In [ ]:
w2v_scores

[(50, 0.887578),
 (48, 0.8868849),
 (52, 0.88353),
 (76, 0.8695435),
 (87, 0.86571234)]

In [ ]:
glove_scores

[(48, 0.9464799931723673),
 (71, 0.9451422398940023),
 (46, 0.938445548483361),
 (6, 0.9329840502694094),
 (45, 0.931776940122057)]

In [ ]:
#문서 내용 조회
def view_txt(index):

  if index > 48:
    root_dir = '/content/drive/MyDrive/일반대화'
    test_file = normal_data
    index -= 48
  else:
    root_dir = '/content/drive/MyDrive/금융감독원'
    test_file = total_data[num_of_train:]

  path = root_dir + '/' + test_file[index]

  targetTXT = open(path, 'r')
  lines = targetTXT.readlines()

  print(lines)

In [ ]:
view_txt(num)

['A.기분 좋은 아침입니다 #@소속# 상담원 #@이름#입니다\n', 'B.여보세요 이삼일 전에 제가 자격증을 잃어버려서요\n', 'B.재발급 신청을 하려고 전화를 드렸었어요\n', 'A.선생님 재발급 신청 서류 관련한 문의십니까\n', 'B.여기 문자 안내받은 대로 제가 준비해서 보내야 되는 거예요\n', 'A.선생님 우편 접수할 예정이십니까\n', 'B.네 우편접수 할 거예요\n', 'A.선생님 신청서와 신분증 사본 한부 반명함 사진 한장이 필요합니다\n', 'B.그래요 뭐가 굉장히 많네요\n', 'A.선생님 신청서 작성 하셨습니까\n', 'B.이제 신청서를 도청에다가 신청 할려구요\n', 'B.제가 교육 받았던데 가서 신청서를 용지를 가지고 왔어요\n', 'A.그럼 가지고 계신 신청서를 작성 해주세요\n', 'A.신분증 사본 한부 반명함 사진 한장 발급비용 이천원 준비해 주시고요\n', 'A.등기료 이천오십원인 총 사천오십원을 준비하시면 돼요\n', 'A.그리고 우체국에서 통상환으로 변경해 주세요\n', 'A.마지막으로 기타서류와 함께 동봉해서 보내주시면 됩니다\n', 'B.알겠습니다 혹시 모르겠으면 다시 전화할게요\n', 'A.감사합니다 상담사 #@이름#이었습니다 즐거운 하루 보내세요']


In [ ]:
view_txt(50)

['A.안녕하세요 #@소속# 상담원 #@이름#입니다\n', 'B.#@소속# 전입지원금이요 그거 금액이 얼마예요\n', 'A.네 선생님 조건마다 조금씩 다른데요 어떤 조건으로 받으십니까\n', 'B.그냥 다 알려줘봐주세요\n', 'A.우선 학생학자금은 최대 팔 회 이십 만원이구요\n', 'A.우리시 소재 기관 단체 및 기업체 임직원이나 귀농인은 삼십 만원입니다\n', 'B.그거 개인 사업자도 포함이죠\n', 'A.네 맞습니다 그리고 고등학생과 대학생의 기숙사비도 지원합니다\n', 'B.그거는 얼마인데요 횟수 제한 있고요\n', 'A.네 최대 팔 회 삼십 만원 이내입니다\n', 'B.더 없나요 그거 말고는\n', 'A.지금 확인 되는 것은 그렇습니다\n', 'A.#@소속# #@전번#으로 문의하시면 자세한 설명 들으실 수 있습니다\n', 'B.네 그러면 거기로 전화해볼게요\n', 'A.네 수월한 업무 처리 되시기 바라겠습니다\n', 'B.예 수고하세요 상담사님\n', 'A.감사합니다 #@소속# #@이름#이었습니다\n', 'B.친절한 상담 감사합니다']


In [ ]:
#두 모델 비교

glove_cnt = 0
w2v_cnt = 0

for idx in range(0, 98):
  target = idx < 49 #보이스피싱 여부

  glove_scores = list(enumerate(glove_cosine_similarities[idx]))
  w2v_scores = list(enumerate(w2v_cosine_similarities[idx]))

  glove_scores = sorted(glove_scores, key = lambda x: x[1], reverse = True)
  glove_scores = glove_scores[1:6]

  w2v_scores = sorted(w2v_scores, key = lambda x: x[1], reverse = True)
  w2v_scores = w2v_scores[1:6]

  for score in glove_scores:
    if target != (score[0] < 49):
      print(idx,"번 Glove: ", glove_scores)
      glove_cnt += 1
      break
  for score in w2v_scores:
    if target != (score[0] < 49):
      print(idx,"번 Word2Vec: ", w2v_scores)
      w2v_cnt += 1
      break

print("\nGlove 오답 횟수: ", glove_cnt, " 정답률: ", (98 - glove_cnt)/98 * 100)
print("Word2Vec 오답 횟수: ", w2v_cnt, " 정답률: ", (98 - w2v_cnt)/98 * 100)


0 번 Glove:  [(2, 0.9144612034931255), (1, 0.8893759500023732), (3, 0.8828564732251293), (89, 0.7821725937825322), (26, 0.7366111704687189)]
0 번 Word2Vec:  [(2, 0.9626539), (1, 0.9594811), (3, 0.9498831), (89, 0.8104457), (29, 0.78268963)]
39 번 Glove:  [(69, 0.943113652337807), (42, 0.9253816627056329), (33, 0.922830676165528), (73, 0.9207902752781564), (35, 0.9191947728561106)]
39 번 Word2Vec:  [(52, 0.8860518), (38, 0.8806588), (42, 0.8722062), (35, 0.8701391), (36, 0.86905634)]
41 번 Glove:  [(24, 0.9477078069777554), (14, 0.9457544460356703), (59, 0.9425894481395694), (7, 0.9415741952803266), (45, 0.9400001459486839)]
41 번 Word2Vec:  [(32, 0.9034395), (59, 0.9004052), (40, 0.8840984), (58, 0.8764512), (31, 0.86848694)]
48 번 Word2Vec:  [(35, 0.95737755), (94, 0.9405162), (13, 0.9351124), (16, 0.9288961), (19, 0.92808175)]
49 번 Glove:  [(48, 0.9426134202382606), (46, 0.9315919288723346), (40, 0.9263508745132054), (83, 0.9262800916850074), (19, 0.9245073604743816)]
49 번 Word2Vec:  [(52, 

실험

In [ ]:
#문서와 단어 간의 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity

tmp = []
cnt = 0
sample_list = ["피해자", "보험", "보이스피싱", "바나나", "사람", "고객님", "펜", "연락", "통장", "입금", "구매", "환불", "달력", "책상", "다음", "책", "종이", "운전", "친구", "가족", "노래", "사랑", "전달", "점심", "전원"]
for sam in sample_list:
  if sam in glove.dictionary.keys():
    tmp.append(glove.word_vectors[glove.dictionary[sam]])
    cnt += 1
    print(sam)
print(cnt)
sample_embedding_list = glove_embedding_list[31:51]
sample_cosine_similarities = cosine_similarity(sample_embedding_list, tmp)

sample_scores = list(enumerate(sample_cosine_similarities[13]))

sample_scores = sorted(sample_scores, key = lambda x: x[1], reverse = True)
sample_scores = sample_scores[1:6]

print(sample_scores)

피해자
보험
보이스피싱
사람
펜
연락
통장
입금
구매
환불
다음
책
종이
운전
친구
가족
사랑
전달
점심
전원
20
[(6, 0.66033992242933), (5, 0.5836716547282954), (3, 0.5488852452414891), (0, 0.5255546064242373), (10, 0.3920335658618733)]


보이스피싱 의심 단어 상위 10개 출력

In [ ]:
#문서끼리 유사도 비교. 입력문서 유사도 상위 단어 10개 뽑기

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

#비교 문서
compare_vec = glove_embedding_list[48]

#입력 문서
input_text = "김민정씨 맞으시죠? 하더니 자기가 검사라고 하면서 제 국민은행이랑 신한은행 계좌가 대포통장으로 사건 현장에서 발견되었다고 함. 그리고 본인확인을 하면서 30만원 입금을 요구했음."

removed_data = []


# 한글 아닌 부분 제거
s = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", input_text)
if s != '':
  removed_data.append(s)

tokenized_data = []
for sentence in tqdm(removed_data):
      tokenized_sentence = okt.morphs(sentence, stem = True) # 토큰화
      tokenized_data = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거

print(tokenized_data)


100%|██████████| 1/1 [00:00<00:00, 110.93it/s]

['김민정', '맞다', '자기', '검사', '국민은행', '이랑', '신한은행', '계좌', '대포통장', '사건', '현장', '발견', '되어다', '함', '본인', '확인', '만원', '입금', '요구']


In [ ]:
#토큰화한 데이터 벡터와 함께 저장
tmp = []
cnt = 0
for sam in tokenized_data:
  if sam == []: continue;
  if sam in glove.dictionary.keys():
    tmp.append([sam, glove.word_vectors[glove.dictionary[sam]]])
    cnt += 1

#단어 벡터 수
print(cnt)

19


In [ ]:
#코사인 유사도 구해서 높은 순서대로 단어와 유사도 출력
answer = []
for i in tmp:
  answer.append([i[0], cos_sim(compare_vec, i[1])])

answer = sorted(answer, key = lambda x: x[1], reverse = True)

#단어와 벡터 출력
print(answer)

[['본인', 0.6899509892777602], ['맞다', 0.6308451652741386], ['확인', 0.6112099921719196], ['입금', 0.5924804072074142], ['계좌', 0.5087297731993747], ['사건', 0.4749439630125671], ['국민은행', 0.46128896245635853], ['만원', 0.3907388403320191], ['되어다', 0.3829123253880537], ['발견', 0.36526398300174645], ['신한은행', 0.3242466735494516], ['이랑', 0.17896262223365494], ['대포통장', 0.17499918709897946], ['요구', 0.12096912174310584], ['현장', 0.07444813092528907], ['검사', 0.004489518670609777], ['자기', -0.5257775647015578], ['함', -0.7138897466600109], ['김민정', -0.8978339863481252]]


In [ ]:
#10위까지 겹치는 단어 없이 (단어, 유사도) 출력
def Sim10(answer):
  answerList = []
  num = 0
  i = -1
  while num < 10:
    i += 1
    if i >= cnt: break;
    if answer[i] in answerList:
      continue
    answerList.append(answer[i])
    num += 1;
  return answerList

단일 문서끼리 유사도 비교

In [ ]:
#입력 문서의 벡터 구하기
inputVec = None

for wordVec in tmp:
  if inputVec is None:
    inputVec = wordVec[1]
  else:
    inputVec = inputVec + wordVec[1]

if inputVec is not None:
  inputVec = inputVec / cnt



In [ ]:
#유사도 비교해서 70이상이면 의심단어 10개 추출
answer_sim = cos_sim(compare_vec, inputVec)

if answer_sim >= 0.7:
  print("보이스피싱이 의심됩니다. ")
  print(answer_sim)
  answerList = Sim10(answer)
  print(answerList)

보이스피싱이 의심됩니다. 
0.721907522961849
[['본인', 0.6899509892777602], ['맞다', 0.6308451652741386], ['확인', 0.6112099921719196], ['입금', 0.5924804072074142], ['계좌', 0.5087297731993747], ['사건', 0.4749439630125671], ['국민은행', 0.46128896245635853], ['만원', 0.3907388403320191], ['되어다', 0.3829123253880537], ['발견', 0.36526398300174645]]


In [ ]:
#비교 문서 벡터 리스트 저장
import pickle

with open('/content/drive/MyDrive/phishing_vec_list.pkl',"wb") as f:
    pickle.dump(glove_embedding_list, f)